## Imports

In [71]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [36]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

Aug. 8 Aug. 14


In [42]:
# Get today's date
today = datetime.now() - timedelta(days=12)
today

datetime.datetime(2025, 8, 3, 15, 15, 43, 428938)

In [45]:
# Get the date one week ago
one_week_ago = today - timedelta(days=6)
one_week_ago

datetime.datetime(2025, 7, 28, 15, 15, 43, 428938)

In [46]:
week_starting = "July 28"
week_ending = "Aug. 3"

In [47]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [48]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [49]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [50]:
df.sort_values(by='PRICE',ascending=False)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
49,PAST SALE,July-29-2025,Condo/Co-op,50 S Pointe Dr #1601,Miami Beach,FL,33139.0,12750000.0,3.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11778282,N,Y,25.767875,-80.132756
85,PAST SALE,July-31-2025,Condo/Co-op,17975 Collins Ave #3801,Sunny Isles Beach,FL,33160.0,10200000.0,5.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11699047,N,Y,25.942332,-80.120166
107,PAST SALE,July-30-2025,Condo/Co-op,17975 Collins #1701,Sunny Isles Beach,FL,33160.0,10150000.0,5.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11693067,N,Y,25.942968,-80.121132
71,PAST SALE,August-1-2025,Condo/Co-op,2900 NE 7th Ave #5102,Miami,FL,33137.0,9050000.0,6.0,8.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11780585,N,Y,25.805004,-80.185632
115,PAST SALE,July-31-2025,Condo/Co-op,17901 Collins Ave #1801,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11653768,N,Y,25.941865,-80.120490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,PAST SALE,July-28-2025,Condo/Co-op,671 NE 195th St Unit 228E,Miami,FL,33179.0,150000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/671-NE-195th-S...,MARMLS,A11826556,N,Y,25.955655,-80.188851
52,PAST SALE,July-30-2025,Condo/Co-op,13500 NE 3rd Ct #114,North Miami,FL,33161.0,140000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13500-NE...,MARMLS,A11787194,N,Y,25.899624,-80.192301
123,PAST SALE,July-29-2025,Condo/Co-op,5300 NW 87th Ave #601,Doral,FL,33178.0,140000.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/5300-NW-87th-A...,MARMLS,A11550430,N,Y,25.820709,-80.338168
66,PAST SALE,July-29-2025,Condo/Co-op,1780 NE 191st St Unit 606-2,Miami,FL,33179.0,132500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1780-NE-191st-...,MARMLS,A11774316,N,Y,25.949923,-80.166133


In [51]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [52]:
df['SOLD DATE'].value_counts()

SOLD DATE
July-31-2025     34
July-28-2025     26
July-29-2025     25
July-30-2025     22
August-1-2025    16
Name: count, dtype: int64

In [53]:
new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'August-1-2025',
          'PROPERTY TYPE':'Condo/Co-op',
          'ADDRESS':'6800 Fisher Island Dr',
          'CITY':'Miami Beach',
          'STATE OR PROVINCE':'FL',
          'ZIP OR POSTAL CODE':'33109',
          'PRICE':17_000_000,
          'BEDS':4,
          'BATHS':4.5,
          'LOCATION':'Palazzo Della Luna',
          'SQUARE FEET':4420,
          'LOT SIZE':None,
          'YEAR BUILT':'2019',
          'DAYS ON MARKET':None,
          '$/SQUARE FEET':3_846,
          'HOA/MONTH':13_712,
          'STATUS':'Sold',
          'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
          'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
          'URL':'https://www.redfin.com/FL/Miami-Beach/6800-Fisher-Island-Dr-6881-33109/unit-6881/home/170955767',
          'SOURCE':'Manual Entry',
          'MLS#':'A11729190',
          'FAVORITE':'N',
          'INTERESTED':'Y',
          'LATITUDE':25.763950636622493,
          'LONGITUDE':-80.13833487686588}

new_row_df = pd.DataFrame(new_row,index=[0])
new_row_df

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,August-1-2025,Condo/Co-op,6800 Fisher Island Dr,Miami Beach,FL,33109,17000000,4,4.5,...,Sold,PLACEHOLDER,PLACEHOLDER,https://www.redfin.com/FL/Miami-Beach/6800-Fis...,Manual Entry,A11729190,N,Y,25.763951,-80.138335


In [54]:
df = pd.concat([df,new_row_df])

In [55]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [56]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [57]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-07-31    34
2025-07-29    25
2025-07-30    22
2025-08-01    17
Name: count, dtype: int64

In [58]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [59]:
df_top_ten = df2.head(10)

In [60]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,2025-08-01,Condo/Co-op,6800 Fisher Island Dr,Miami Beach,FL,33109,17000000.0,4.0,4.5,...,Sold,PLACEHOLDER,PLACEHOLDER,https://www.redfin.com/FL/Miami-Beach/6800-Fis...,Manual Entry,A11729190,N,Y,25.763951,-80.138335
49,PAST SALE,2025-07-29,Condo/Co-op,50 S Pointe Dr #1601,Miami Beach,FL,33139.0,12750000.0,3.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11778282,N,Y,25.767875,-80.132756
85,PAST SALE,2025-07-31,Condo/Co-op,17975 Collins Ave #3801,Sunny Isles Beach,FL,33160.0,10200000.0,5.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11699047,N,Y,25.942332,-80.120166
107,PAST SALE,2025-07-30,Condo/Co-op,17975 Collins #1701,Sunny Isles Beach,FL,33160.0,10150000.0,5.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11693067,N,Y,25.942968,-80.121132
71,PAST SALE,2025-08-01,Condo/Co-op,2900 NE 7th Ave #5102,Miami,FL,33137.0,9050000.0,6.0,8.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11780585,N,Y,25.805004,-80.185632
115,PAST SALE,2025-07-31,Condo/Co-op,17901 Collins Ave #1801,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11653768,N,Y,25.941865,-80.120490
58,PAST SALE,2025-07-29,Condo/Co-op,6000 Island Blvd #1703,Aventura,FL,33160.0,3525000.0,4.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11782589,N,Y,25.942761,-80.135433
97,PAST SALE,2025-07-29,Condo/Co-op,400 Alton Rd #1201,Miami Beach,FL,33139.0,2880000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-Alto...,MARMLS,A11706643,N,Y,25.772715,-80.140416
36,PAST SALE,2025-07-31,Condo/Co-op,1425 Brickell Ave Unit 41B,Miami,FL,33131.0,2373333.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,Beaches MLS,F10500746,N,Y,25.759083,-80.191797
98,PAST SALE,2025-07-31,Condo/Co-op,18671 Collins Ave #1602,Sunny Isles Beach,FL,33160.0,1950000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11712892,N,Y,25.948748,-80.119749


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [61]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)
        time.sleep(0.2)
        response.raise_for_status()

        # If .json() returns None, fall back on empty dict
        data = response.json() or {}

        # Drill down with or {} at each step to avoid None.get errors
        panel_info = (data.get('data') or {}) \
            .get('mainHouseInfoPanelInfo') or {}
        main_info = panel_info.get('mainHouseInfo') or {}

        # Now safe to pull lists
        listing_agents = main_info.get('listingAgents') or []
        buying_agents  = main_info.get('buyingAgents')  or []

        # Defaults
        list_agent_name_1 = list_broker_name_1 = None
        list_agent_name_2 = list_broker_name_2 = None
        buy_agent_name_1  = buy_broker_name_1  = None
        buy_agent_name_2  = buy_broker_name_2  = None

        # Extract up to two listing agents
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')

        # Extract up to two buying agents
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }


In [62]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [63]:
response_df = pd.DataFrame(response_list)

In [64]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [65]:
df_top_ten = merged_df

## Current Week's Values

In [67]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

98


In [68]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,240,658


In [69]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$604


In [70]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$121,584,533


In [23]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [38]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:


 Miami-Dade


Input Previous Week Condo Sales Total (number units sold):
88
-----------
Input Previous Week Condo Average Sales Price:
1259057
-----------
Input Previous Week Condo Average PSF:
635
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
110797028.0


In [39]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,88,1259057,635,110797028.0


In [40]:
last_week['Last Week Total Dollar Volume'].iloc[0]

110797028.0

In [42]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           97             1078191          570                    104584533.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [43]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [44]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [46]:
pd.set_option('display.max_columns',None)

In [47]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-29,Condo/Co-op,50 S Pointe Dr #1601,Miami Beach,FL,33139.0,12750000.0,3.0,4.0,CONTINUUM ON SOUTH BEACH,3497.0,NaN,2008.0,NaN,3646.0,17950.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11778282,N,Y,25.767875,-80.132756,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Julian Johnston,The Corcoran Group,None,None,1,orange


In [48]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-29,Condo/Co-op,50 S Pointe Dr #1601,Miami Beach,FL,33139.0,12750000.0,3.0,4.0,CONTINUUM ON SOUTH BEACH,3497.0,NaN,2008.0,NaN,3646.0,17950.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11778282,N,Y,25.767875,-80.132756,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Julian Johnston,The Corcoran Group,None,None,1,orange
1,PAST SALE,2025-07-31,Condo/Co-op,17975 Collins Ave #3801,Sunny Isles Beach,FL,33160.0,10200000.0,5.0,6.5,Estates at Acqualina North,4780.0,NaN,2022.0,NaN,2134.0,8985.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11699047,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adriana Vargas Hernandez,Acqualina Realty,None,None,Adriana Vargas Hernandez,Acqualina Realty,Adriana Vargas Hernandez,Acqualina Realty,2,blue
2,PAST SALE,2025-07-30,Condo/Co-op,17975 Collins #1701,Sunny Isles Beach,FL,33160.0,10150000.0,5.0,6.5,The Estates at Acqualina,4780.0,NaN,2023.0,NaN,2123.0,9947.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11693067,N,Y,25.942968,-80.121132,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Yansy Checa,Douglas Elliman,None,None,Susan Miner,Celebrity Advisors,None,None,3,blue
3,PAST SALE,2025-08-01,Condo/Co-op,2900 NE 7th Ave #5102,Miami,FL,33137.0,9050000.0,6.0,8.0,BISCAYNE BEACH CONDO,8434.0,NaN,2017.0,NaN,1073.0,11107.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11780585,N,Y,25.805004,-80.185632,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bill Hernandez,Douglas Elliman,Bryan Sereny,Douglas Elliman,Cecilia Cordova,Redfin Corporation,None,None,4,blue
4,PAST SALE,2025-07-31,Condo/Co-op,17901 Collins Ave #1801,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,Estates at Acqualina,3840.0,NaN,2022.0,NaN,1953.0,8501.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11653768,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,5,blue
5,PAST SALE,2025-07-29,Condo/Co-op,6000 Island Blvd #1703,Aventura,FL,33160.0,3525000.0,4.0,3.5,BELLAMARE @ WILLIAMS ISLA,3224.0,NaN,2005.0,NaN,1093.0,4445.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11782589,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Richard Goihman,Douglas Elliman,None,None,Richard Goihman,Douglas Elliman,None,None,6,blue
6,PAST SALE,2025-07-29,Condo/Co-op,400 Alton Rd #1201,Miami Beach,FL,33139.0,2880000.0,2.0,2.5,MURANO GRANDE AT PORTOFIN,2003.0,NaN,2003.0,NaN,1438.0,1900.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-Alto...,MARMLS,A11706643,N,Y,25.772715,-80.140416,https://www.redfin.com/FL/Miami-Beach/400-Alto...,Nancy Batchelor,"Compass Florida, LLC.",Andrew Holtz,"Compass Florida, LLC.",Marc Russo,"Income Real Estate, Inc.",None,None,7,blue
7,PAST SALE,2025-07-31,Condo/Co-op,1425 Brickell Ave Unit 41B,Miami,FL,33131.0,2373333.0,2.0,2.5,Four Seasons Res.,2400.0,NaN,2003.0,NaN,989.0,2966.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,Beaches MLS,F10500746,N,Y,25.759083,-80.191797,https://www.redfin.com/FL/Miami/1425-Brickell-...,Camilo Perez,Home Sold Realty LLC,None,None,Diego Ramos,"Compass Florida, LLC",None,None,8,blue
8,PAST SALE,2025-07-31,Condo/Co-op,18671 Collins Ave #1602,Sunny Isles Beach,FL,33160.0,1950000.0,3.0,3.5,Millennium Condo,2210.0,NaN,1999.0,NaN,882.0,1936.0

In [50]:
%store -r map_box_api_key

In [51]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=10.5,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [52]:
m.save('index.html')

## Data snagger

In [53]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [54]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-29,Condo/Co-op,50 S Pointe Dr #1601,Miami Beach,FL,33139.0,12750000.0,3.0,4.0,CONTINUUM ON SOUTH BEACH,3497.0,NaN,2008.0,NaN,3646.0,17950.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11778282,N,Y,25.767875,-80.132756,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Julian Johnston,The Corcoran Group,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [55]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_08042025


# CREATE TEMPLATE 

In [56]:
muni_set = set(df_top_ten['CITY'])

In [57]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [58]:
df_top_ten.reset_index(inplace=True,drop=True)

In [59]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [60]:
top_sale

'CONTINUUM ON SOUTH BEACH at 50 S Pointe Dr #1601 in Miami Beach'

In [61]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [62]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [63]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-29,Condo/Co-op,50 S Pointe Dr #1601,Miami Beach,FL,33139.0,12750000.0,3.0,4.0,CONTINUUM ON SOUTH BEACH,3497.0,NaN,2008.0,NaN,3646.0,17950.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11778282,N,Y,25.767875,-80.132756,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Julian Johnston,The Corcoran Group,None,None,1,orange,CONTINUUM ON SOUTH BEACH at 50 S Pointe Dr #16...
1,PAST SALE,2025-07-31,Condo/Co-op,17975 Collins Ave #3801,Sunny Isles Beach,FL,33160.0,10200000.0,5.0,6.5,Estates at Acqualina North,4780.0,NaN,2022.0,NaN,2134.0,8985.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11699047,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adriana Vargas Hernandez,Acqualina Realty,None,None,Adriana Vargas Hernandez,Acqualina Realty,Adriana Vargas Hernandez,Acqualina Realty,2,blue,Estates at Acqualina North at 17975 Collins Av...
2,PAST SALE,2025-07-30,Condo/Co-op,17975 Collins #1701,Sunny Isles Beach,FL,33160.0,10150000.0,5.0,6.5,The Estates at Acqualina,4780.0,NaN,2023.0,NaN,2123.0,9947.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11693067,N,Y,25.942968,-80.121132,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Yansy Checa,Douglas Elliman,None,None,Susan Miner,Celebrity Advisors,None,None,3,blue,The Estates at Acqualina at 17975 Collins #170...
3,PAST SALE,2025-08-01,Condo/Co-op,2900 NE 7th Ave #5102,Miami,FL,33137.0,9050000.0,6.0,8.0,BISCAYNE BEACH CONDO,8434.0,NaN,2017.0,NaN,1073.0,11107.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11780585,N,Y,25.805004,-80.185632,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bill Hernandez,Douglas Elliman,Bryan Sereny,Douglas Elliman,Cecilia Cordova,Redfin Corporation,None,None,4,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #5102 ...
4,PAST SALE,2025-07-31,Condo/Co-op,17901 Collins Ave #1801,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,Estates at Acqualina,3840.0,NaN,2022.0,NaN,1953.0,8501.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11653768,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,5,blue,Estates at Acqualina at 17901 Collins Ave #180...
5,PAST SALE,2025-07-29,Condo/Co-op,6000 Island Blvd #1703,Aventura,FL,33160.0,3525000.0,4.0,3.5,BELLAMARE @ WILLIAMS ISLA,3224.0,NaN,2005.0,NaN,1093.0,4445.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11782589,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Richard Goihman,Douglas Elliman,None,None,Richard Goihman,Douglas Elliman,None,None,6,blue,BELLAMARE @ WILLIAMS ISLA at 6000 Island Blvd ...
6,PAST SALE,2025-07-29,Condo/Co-op,400 Alton Rd #1201,Miami Beach,FL,33139.0,2880000.0,2.0,2.5,MURANO GRANDE AT PORTOFIN,2003.0,NaN,2003.0,NaN,1438.0,1900.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-Alto...,MARMLS,A11706643,N,Y,25.772715,-80.140416,https://www.redfin.com/FL/Miami-Beach/400-Alto...,Nancy Batchelor,"Compass Florida, LLC.",Andrew Holtz,"Compass Florida, LLC.",Marc Russo,"Income Real Estate, Inc.",None,None,7,blue,MURANO GRANDE AT PORTOFIN at 400 Alton Rd #120...
7,PAST SALE,2025-07-31,Condo/Co-op,1425 Brickell Ave Unit 41B,Miami,FL,33131.0,2373333.0,2.0,2.5,Four Seasons Res.,2400.0,NaN,2003.0,NaN,989.0,2966.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Bri

In [64]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: CONTINUUM ON SOUTH BEACH closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,900,000 to $12,750,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Aventura, Sunny Isles Beach, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 97 condo sales totaling $104,584,533 million from July 28 to Aug. 3. The previous week, brokers closed 88 condo sales totaling $110,797,028.

Last week’s units sold for an average of $1,078,191, lower than the $1,259,057 average price from the

In [65]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [66]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [67]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [68]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-29,Condo/Co-op,50 S Pointe Dr #1601,Miami Beach,FL,33139.0,12750000.0,3.0,4.0,CONTINUUM ON SOUTH BEACH,3497.0,NaN,2008.0,NaN,3646.0,17950.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11778282,N,Y,25.767875,-80.132756,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Julian Johnston,The Corcoran Group,None,None,1,orange,CONTINUUM ON SOUTH BEACH at 50 S Pointe Dr #16...
1,PAST SALE,2025-07-31,Condo/Co-op,17975 Collins Ave #3801,Sunny Isles Beach,FL,33160.0,10200000.0,5.0,6.5,Estates at Acqualina North,4780.0,NaN,2022.0,NaN,2134.0,8985.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11699047,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adriana Vargas Hernandez,Acqualina Realty,None,None,Adriana Vargas Hernandez,Acqualina Realty,Adriana Vargas Hernandez,Acqualina Realty,2,blue,Estates at Acqualina North at 17975 Collins Av...
2,PAST SALE,2025-07-30,Condo/Co-op,17975 Collins #1701,Sunny Isles Beach,FL,33160.0,10150000.0,5.0,6.5,The Estates at Acqualina,4780.0,NaN,2023.0,NaN,2123.0,9947.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11693067,N,Y,25.942968,-80.121132,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Yansy Checa,Douglas Elliman,None,None,Susan Miner,Celebrity Advisors,None,None,3,blue,The Estates at Acqualina at 17975 Collins #170...
3,PAST SALE,2025-08-01,Condo/Co-op,2900 NE 7th Ave #5102,Miami,FL,33137.0,9050000.0,6.0,8.0,BISCAYNE BEACH CONDO,8434.0,NaN,2017.0,NaN,1073.0,11107.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11780585,N,Y,25.805004,-80.185632,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bill Hernandez,Douglas Elliman,Bryan Sereny,Douglas Elliman,Cecilia Cordova,Redfin Corporation,None,None,4,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #5102 ...
4,PAST SALE,2025-07-31,Condo/Co-op,17901 Collins Ave #1801,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,Estates at Acqualina,3840.0,NaN,2022.0,NaN,1953.0,8501.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11653768,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,5,blue,Estates at Acqualina at 17901 Collins Ave #180...
5,PAST SALE,2025-07-29,Condo/Co-op,6000 Island Blvd #1703,Aventura,FL,33160.0,3525000.0,4.0,3.5,BELLAMARE @ WILLIAMS ISLA,3224.0,NaN,2005.0,NaN,1093.0,4445.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11782589,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Richard Goihman,Douglas Elliman,None,None,Richard Goihman,Douglas Elliman,None,None,6,blue,BELLAMARE @ WILLIAMS ISLA at 6000 Island Blvd ...
6,PAST SALE,2025-07-29,Condo/Co-op,400 Alton Rd #1201,Miami Beach,FL,33139.0,2880000.0,2.0,2.5,MURANO GRANDE AT PORTOFIN,2003.0,NaN,2003.0,NaN,1438.0,1900.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-Alto...,MARMLS,A11706643,N,Y,25.772715,-80.140416,https://www.redfin.com/FL/Miami-Beach/400-Alto...,Nancy Batchelor,"Compass Florida, LLC.",Andrew Holtz,"Compass Florida, LLC.",Marc Russo,"Income Real Estate, Inc.",None,None,7,blue,MURANO GRANDE AT PORTOFIN at 400 Alton Rd #120...
7,PAST SALE,2025-07-31,Condo/Co-op,1425 Brickell Ave Unit 41B,Miami,FL,33131.0,2373333.0,2.0,2.5,Four Seasons Res.,2400.0,NaN,2003.0,NaN,989.0,2966.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Bri

In [69]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: CONTINUUM ON SOUTH BEACH closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,900,000 to $12,750,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Aventura, Sunny Isles Beach, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 97 condo sales totaling $104,584,533 million from July 28 to Aug. 3. The previous week, brokers closed 88 condo sales totaling $110,797,028.

Last week’s units sold for an average of $1,078,191, lower than the $1,259,057 average price from the

In [77]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami-Beach/50-S-Pointe-Dr-33139/unit-1601/home/43405108


In [78]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/17975-Collins-Ave-33160/unit-3801/home/193685660


In [79]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/2655-S-Bayshore-Dr-33133/unit-1406/home/192233250


In [80]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Sunny-Isles-Beach/18671-Collins-Ave-33160/unit-1602/home/43254279


In [81]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/50-S-Pointe-Dr-33139/unit-1601/home/43405108


In [82]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/18671-Collins-Ave-33160/unit-1602/home/43254279


In [83]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-29,Condo/Co-op,50 S Pointe Dr #1601,Miami Beach,FL,33139.0,12750000.0,3.0,4.0,CONTINUUM ON SOUTH BEACH,3497.0,NaN,2008.0,NaN,3646.0,17950.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11778282,N,Y,25.767875,-80.132756,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Julian Johnston,The Corcoran Group,None,None,1,orange,CONTINUUM ON SOUTH BEACH at 50 S Pointe Dr #16...
1,PAST SALE,2025-07-31,Condo/Co-op,17975 Collins Ave #3801,Sunny Isles Beach,FL,33160.0,10200000.0,5.0,6.5,Estates at Acqualina North,4780.0,NaN,2022.0,NaN,2134.0,8985.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11699047,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adriana Vargas Hernandez,Acqualina Realty,None,None,Adriana Vargas Hernandez,Acqualina Realty,Adriana Vargas Hernandez,Acqualina Realty,2,blue,Estates at Acqualina North at 17975 Collins Av...
2,PAST SALE,2025-07-30,Condo/Co-op,17975 Collins #1701,Sunny Isles Beach,FL,33160.0,10150000.0,5.0,6.5,The Estates at Acqualina,4780.0,NaN,2023.0,NaN,2123.0,9947.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11693067,N,Y,25.942968,-80.121132,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Yansy Checa,Douglas Elliman,None,None,Susan Miner,Celebrity Advisors,None,None,3,blue,The Estates at Acqualina at 17975 Collins #170...
3,PAST SALE,2025-08-01,Condo/Co-op,2900 NE 7th Ave #5102,Miami,FL,33137.0,9050000.0,6.0,8.0,BISCAYNE BEACH CONDO,8434.0,NaN,2017.0,NaN,1073.0,11107.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11780585,N,Y,25.805004,-80.185632,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bill Hernandez,Douglas Elliman,Bryan Sereny,Douglas Elliman,Cecilia Cordova,Redfin Corporation,None,None,4,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #5102 ...
4,PAST SALE,2025-07-31,Condo/Co-op,17901 Collins Ave #1801,Sunny Isles Beach,FL,33160.0,7500000.0,4.0,5.0,Estates at Acqualina,3840.0,NaN,2022.0,NaN,1953.0,8501.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11653768,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,5,blue,Estates at Acqualina at 17901 Collins Ave #180...
5,PAST SALE,2025-07-29,Condo/Co-op,6000 Island Blvd #1703,Aventura,FL,33160.0,3525000.0,4.0,3.5,BELLAMARE @ WILLIAMS ISLA,3224.0,NaN,2005.0,NaN,1093.0,4445.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11782589,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Richard Goihman,Douglas Elliman,None,None,Richard Goihman,Douglas Elliman,None,None,6,blue,BELLAMARE @ WILLIAMS ISLA at 6000 Island Blvd ...
6,PAST SALE,2025-07-29,Condo/Co-op,400 Alton Rd #1201,Miami Beach,FL,33139.0,2880000.0,2.0,2.5,MURANO GRANDE AT PORTOFIN,2003.0,NaN,2003.0,NaN,1438.0,1900.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-Alto...,MARMLS,A11706643,N,Y,25.772715,-80.140416,https://www.redfin.com/FL/Miami-Beach/400-Alto...,Nancy Batchelor,"Compass Florida, LLC.",Andrew Holtz,"Compass Florida, LLC.",Marc Russo,"Income Real Estate, Inc.",None,None,7,blue,MURANO GRANDE AT PORTOFIN at 400 Alton Rd #120...
7,PAST SALE,2025-07-31,Condo/Co-op,1425 Brickell Ave Unit 41B,Miami,FL,33131.0,2373333.0,2.0,2.5,Four Seasons Res.,2400.0,NaN,2003.0,NaN,989.0,2966.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Bri

## Time on Market Calculator

In [72]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 1, 15) ## List (Earlier) date
date2 = datetime(2025, 8, 1) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

198


## Clean CSV for Datawrapper Chart

In [87]:
chart_df = df_top_ten

In [88]:
chart_df = chart_df.fillna(" ")

In [89]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [90]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [91]:
chart_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME', 'LISTING AGENTS',
       'BUYING AGENTS'],
      dtype='object')

In [92]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [93]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [94]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [95]:
chart_df['PPSF'].astype(float)

0    3646.0
1    2134.0
2    2123.0
3    1073.0
4    1953.0
5    1093.0
6    1438.0
7     989.0
8     882.0
9    1923.0
Name: PPSF, dtype: float64

In [96]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [97]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [98]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-07-29,CONTINUUM ON SOUTH BEACH,50 S Pointe Dr #1601 in Miami Beach,"$12,750,000","3,497","$3,646",2008,Michael Duchon The Corcoran Group Eloy Carmena...,Julian Johnston The Corcoran Group
1,2025-07-31,Estates at Acqualina North,17975 Collins Ave #3801 in Sunny Isles Beach,"$10,200,000","4,780","$2,134",2022,Adriana Vargas Hernandez Acqualina Realty,Adriana Vargas Hernandez Acqualina Realty Adri...
2,2025-07-30,The Estates at Acqualina,17975 Collins #1701 in Sunny Isles Beach,"$10,150,000","4,780","$2,123",2023,Yansy Checa Douglas Elliman,Susan Miner Celebrity Advisors
3,2025-08-01,BISCAYNE BEACH CONDO,2900 NE 7th Ave #5102 in Miami,"$9,050,000","8,434","$1,073",2017,Bill Hernandez Douglas Elliman Bryan Sereny Do...,Cecilia Cordova Redfin Corporation
4,2025-07-31,Estates at Acqualina,17901 Collins Ave #1801 in Sunny Isles Beach,"$7,500,000","3,840","$1,953",2022,Ryan Mendell Maxwell E Realty Inc,Yansy Checa Douglas Elliman
5,2025-07-29,BELLAMARE @ WILLIAMS ISLA,6000 Island Blvd #1703 in Aventura,"$3,525,000","3,224","$1,093",2005,Richard Goihman Douglas Elliman,Richard Goihman Douglas Elliman
6,2025-07-29,MURANO GRANDE AT PORTOFIN,400 Alton Rd #1201 in Miami Beach,"$2,880,000","2,003","$1,438",2003,"Nancy Batchelor Compass Florida, LLC. Andrew H...","Marc Russo Income Real Estate, Inc."
7,2025-07-31,Four Seasons Res.,1425 Brickell Ave Unit 41B in Miami,"$2,373,333","2,400",$989,2003,Camilo Perez Home Sold Realty LLC,"Diego Ramos Compass Florida, LLC"
8,2025-07-31,Millennium Condo,18671 Collins Ave #1602 in Sunny Isles Beach,"$1,950,000","2,210",$882,1999,,
9,2025-08-01,MR C,2655 S Bayshore #1406 in Miami,"$1,900,000",988,"$1,923",2024,Ysabel Sanabria Florida Capital Realty,Arina Feeney Douglas Elliman


In [99]:
csv_date_string = today.strftime("%m_%d_%Y")

In [100]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [101]:
chart_df.columns

Index(['SOLD DATE', 'LOCATION', 'ADDRESS', 'PRICE', 'SQUARE FEET', 'PPSF',
       'YEAR BUILT', 'LISTING AGENTS', 'BUYING AGENTS'],
      dtype='object')